In [33]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Importing libraries

In [34]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from apyori import apriori

## Loading Dataset

In [35]:
df = pd.read_csv('/home/tuandinh/Desktop/MachineLearning/Association Rule Mining/Groceries_dataset.csv', parse_dates=['Date'])
df.head()

/home/tuandinh/Desktop/MachineLearning/tuanenv/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '21-07-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/home/tuandinh/Desktop/MachineLearning/tuanenv/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '19-09-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/home/tuandinh/Desktop/MachineLearning/tuanenv/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '14-02-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/home/tuandinh/Desktop/MachineLearning/tuanenv/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1063: UserWarning:

Parsing '23-12-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/home/tuandi

,Member_number,Date,itemDescription
0,1808,2015-07-21,tropical fruit
1,2552,2015-05-01,whole milk
2,2300,2015-09-19,pip fruit
3,1187,2015-12-12,other vegetables
4,3037,2015-01-02,whole milk


### Any null values

In [36]:
df.isnull().any()

Member_number      False
Date               False
itemDescription    False
dtype: bool

### Total Products

In [37]:
all_products = df['itemDescription'].unique()
print("Total products: {}".format(len(all_products)))

Total products: 167


## Top 10 frequently sold products

In [38]:
def ditribution_plot(x,y,name=None,xaxis=None,yaxis=None):
    fig = go.Figure([
        go.Bar(x=x, y=y)
    ])

    fig.update_layout(
        title_text=name,
        xaxis_title=xaxis,
        yaxis_title=yaxis
    )
    fig.show()

In [39]:
x = df['itemDescription'].value_counts()
x = x.sort_values(ascending = False) 
x = x[:10]

ditribution_plot(x=x.index, y=x.values, yaxis="Count", xaxis="Products")

### One-hot representation of products purchased

In [40]:
one_hot = pd.get_dummies(df['itemDescription'])
df.drop('itemDescription', inplace=True, axis=1)
df = df.join(one_hot)
df.head()

,Member_number,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1808,2015-07-21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2552,2015-05-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2300,2015-09-19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1187,2015-12-12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3037,2015-01-02,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Transactions
Note: if a customer bought multiple products on same day, We will consider it one transaction 

In [41]:
records = df.groupby(["Member_number","Date"])[all_products[:]].apply(sum)
records = records.reset_index()[all_products]

In [42]:
## Replacing non-zero values with product names
def get_Pnames(x):
    for product in all_products:
        if x[product] > 0:
            x[product] = product
    return x

records = records.apply(get_Pnames, axis=1)
records.head()

,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,...,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
0,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
print("total transactions: {}".format(len(records)))

total transactions: 14963


In [44]:
## Removing zeros
x = records.values
x = [sub[~(sub == 0)].tolist() for sub in x if sub[sub != 0].tolist()]
transactions = x

### Example transactions

In [45]:
transactions[0:10]

[['whole milk', 'pastry', 'salty snack'],
 ['whole milk', 'yogurt', 'sausage', 'semi-finished bread'],
 ['soda', 'pickled vegetables'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['whole milk', 'rolls/buns', 'sausage'],
 ['whole milk', 'soda'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['frankfurter', 'curd'],
 ['beef', 'white bread']]

## Association Rules

In [46]:
rules = apriori(transactions,min_support=0.00030,min_confidance=0.05,min_lift=3,min_length=2,target="rules")
association_results = list(rules)

In [47]:
for item in association_results:

    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    print("Support: " + str(item[1]))

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: frozen fish -> specialty chocolate
Support: 0.0003341575887188398
Confidence: 0.049019607843137254
Lift: 3.0689556157190907
Rule: fruit/vegetable juice -> liver loaf
Support: 0.00040098910646260775
Confidence: 0.011787819253438114
Lift: 3.52762278978389
Rule: pickled vegetables -> ham
Support: 0.0005346521419501437
Confidence: 0.03125
Lift: 3.4895055970149254
Rule: roll products  -> meat
Support: 0.0003341575887188398
Confidence: 0.019841269841269844
Lift: 3.620547812620984
Rule: salt -> misc. beverages
Support: 0.0003341575887188398
Confidence: 0.0211864406779661
Lift: 3.5619405827461437
Rule: spread cheese -> misc. beverages
Support: 0.0003341575887188398
Confidence: 0.0211864406779661
Lift: 3.170127118644068
Rule: seasonal products -> soups
Support: 0.0003341575887188398
Confidence: 0.04716981132075471
Lift: 14.704205974842766
Rule: spread cheese -> sugar
Support: 0.00040098910646260775
Confidence: 0.06
Lift: 3.3878490566037733
Rule: bottled beer -> butter
Support: 0.000334157